# Sigma_density: M Dwarfs with Spectroscopic Constraints

Calculating density constraints for M Dwarf KOIs with spectroscopic constraints on properties.

1. From spectroscopy, we know Temperature, Mass, and Radius.
2. Which isochrones (stellar models, with Temperature, Mass, and Radius as parameters) fit above star?
3. Now take into account GAIA distance for the star. From this, we know the Luminosity.
4. Look back at stellar models - which isochrone fits the Temperature, Mass Radius, and Luminority (from GAIA)?

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from astropy.table import Table

plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [62]:
headers = ['KOI', 'KIC', 'Teff', 'ETeff', 'eTeff', 'Fe/H', 'e_Fe/H', 'M/H', 'e_M/H', 'SpType', 'Mstar', 'e_Mstar', 'Rstar', 'e_Rstar', 'Dis', 'Fe/H-T', 'e_Fe/H-T', 'Fe/H-M', 'e_Fe/H-M', 'M/H-M', 'e_M/H-M']

For now, removed any rows with missing data and saved them into "muirhead_data_no_missing_data.txt".

In [63]:
muirhead_data = pd.read_csv("/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/muirhead_data_no_missing_data.txt", sep=" ")

In [64]:
isochrones = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/isochrones_sdss_spitzer_lowmass.dat', sep='\s\s+', engine='python')

#### First, let's work with muirhead_data[0]: KIC 5868793.

1. Take Teff, Mass, and Radius.
2. Which isochrones fit these Teff, Mass, and Radius?

#### Reading Gaia-Kepler data (luminosities)

In [13]:
dat = Table.read('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Kepler-Gaia/kepler_dr2_4arcsec.fits', format='fits')
df = dat.to_pandas()

/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_result_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_modality_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [168]:
muirhead_gaia = df[df['kepid'].isin(list(muirhead_data.KIC))]
muirhead_gaia = muirhead_gaia.reset_index()
muirhead_gaia.rename(columns={"index": "KIC"})

#muirhead_comb: planet hosts with spectroscopic data + Gaia/Kepler data in one table
muirhead_comb = pd.concat([muirhead_data, muirhead_gaia], axis=1)

In [169]:
test_planet = muirhead_comb.loc[muirhead_comb['KIC'] == 10388286.0]

In [170]:
test_planet.lum_val

29    0.100968
Name: lum_val, dtype: float32

In [171]:
#muirhead_comb["KIC"]

In [172]:
#muirhead_comb["lum_val"]

In [173]:
test_planet

,KOI,KIC,Teff,ETeff,eTeff,Fe/H,e_Fe/H,M/H,e_M/H,SpType,Mstar,e_Mstar,Rstar,e_Rstar,Dis,Fe/H-T,e_Fe/H-T,Fe/H-M,e_Fe/H-M,M/H-M,e_M/H-M,index,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,epoch_photometry_url,kepid,ra_kic,dec_kic,r_est,r_lo,r_hi,r_length_prior,r_result_flag,r_modality_flag,tm_designation,kepmag,teff,teff_err1,teff_err2,teff_prov,logg,logg_err1,logg_err2,logg_prov,feh,feh_err1,feh_err2,feh_prov,radius,radius_err1,radius_err2,mass,mass_err1,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist
29,596,10388286.0,3670.0,47.0,62.0,0.01,0.13,0.0,0.12,M1V,0.49,0.04,0.46,0.04,117.0,0.3,0.13,0.13,0.12,0.08,0.12,66848,1635721458409799680,b'Gaia DR2 2076782134064256256',2076782134064256256,1264990159,2015.5,296.876961,0.027556,41.080451,0.032424,4.51892,0.034692,130.259155,21.749907,0.05766,-26.387574,0.056427,0.079808,0.149847,-0.144091,-0.125035,0.362977,-0.194809,-0.298952,-0.260402,-0.142441,0.031841,246,0,246,0,11.528055,591.813843,0.262565,12.748386,31,False,9.85656,1.391522,0.008355,0.005119,28,16,0.063,0,30,False,262,26631.750546,8.000736,3328.662842,14.624866,29,7519.437573,16.148043,465.656281,15.660925,30,28795.030695,40.585068,709.498169,13.613626,1.363578,0,2.047298,1.036058,1.01124,NaN,NaN,0,NaN,NaN,NaN,b'NOT_AVAILABLE',75.343661,7.882406,313.788457,60.498711,100001,3760.0,3675.75,3917.75,1.7585,1.5099,2.056,0.8753,0.7409,1.0296,200111,0.748805,0.689717,0.783525,0.100968,0.099149,0.102788,b'http://geadata.esac.esa.int/data-server/data...,b' ',5809954,296.87683,41.080582,219.904954,218.227318,221.608117,2066.41772,1.0,1.0,b'2MASS J19473044+4104500',14.645,3645,72.0,-90.0,b'SPE70',4.797,0.071,-0.038,b'PHO16',-0.08,0.15,-0.15,b'SPE70',0.449,0.04,-0.061,0.46,0.043,-0.059,b'DSEP',0,1,2,12.29,11.631,11.452,b'cand',0.065163


In [174]:
def fit_isochrone(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia data for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ONLY ON SPECTROSCOPY.
       """            
    
    iso_fits_final = list()
    
    #test each star in spectroscopy sample:
    #for i in tqdm(range(len(muirhead_comb))):
    for i in tqdm(range(1)):

        iso_fits = pd.DataFrame() 

        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]

        #test each stellar model to see if it falls within error bars:
        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])

        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]

        iso_fits_final.append(iso_fits)

    return iso_fits_final


In [175]:
#isos = fit_isochrone(muirhead_comb, isochrones)

In [176]:
#isochrones['logg']

In [177]:
#for i in range(len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str([i]) + ".csv")

In [178]:
#for i in range(12, len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str(isos[i]['KIC'].iloc[0]) + ".csv")

In [179]:
def fit_isochrone_lum(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ON SPECTROSCOPY AND GAIA LUMINOSITY.
       """    
    
    iso_fits_final = list()
    
    #for i in tqdm(range(len(muirhead_comb))):
    for i in range(1):
        
        iso_fits = pd.DataFrame() 
    
#         Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
#         Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
#         Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]
#         lum_range = [data.lum_val[i]-data.lum_percentile_lower[i], data.lum_val[i]+data.lum_percentile_lower[i]]

        Teff_range = [float(data.Teff)-float(data.eTeff), float(data.Teff)+float(data.ETeff)]
        Mstar_range = [float(data.Mstar)-float(data.e_Mstar), float(data.Mstar)+float(data.e_Mstar)]
        Rstar_range = [float(data.Rstar)-float(data.e_Rstar), float(data.Rstar)+float(data.e_Rstar)]
        lum_range = [float(data.lum_val)-float(data.lum_percentile_lower), float(data.lum_val)+float(data.lum_percentile_lower)]

        print(Teff_range)
        print(Mstar_range)
        print(Rstar_range)
        print(lum_range)
        
        for j in tqdm(range(len(isochrones))):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1] and lum_range[0] < 10**isochrones.logl_ls[j] < lum_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])
        
        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]
        
        iso_fits_final.append(iso_fits)
            
    return iso_fits_final

In [152]:
iso_lums = fit_isochrone_lum(test_planet, isochrones)


  0%|          | 2739/738479 [00:00<00:26, 27373.64it/s]

[3608.0, 3717.0]
[0.45, 0.53]
[0.42000000000000004, 0.5]
[0.0018194913864135742, 0.2001165747642517]


100%|██████████| 738479/738479 [00:54<00:00, 13570.96it/s]


In [180]:
for i in range(len(iso_lums)):
   try:
       iso_lums[i].to_csv("jan29_21_iso_lums_" + '10388286' + ".csv")
   except IndexError:
       pass

In [181]:
isodf = pd.read_csv(r'/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/photoeccentric/notebooks/jan29_21_iso_lums_10388286.csv')

In [182]:
mstar = isodf["mstar"].mean()
mstar_err = isodf["mstar"].std()

rstar = isodf["radius"].mean()
rstar_err = isodf["radius"].std()

### Calculate rho

In [183]:
#Let's just do 1 star for now
ntargs = 1

In [184]:
# def find_density_dist_symmetric(ntargs, masses, masserr, radii, raderr):
#     """Gets symmetric stellar density distribution for stars.
#     Symmetric stellar density distribution = Gaussian with same sigma on each end.

#     Parameters
#     ----------
#     ntargs: int
#         Number of stars to get distribution for
#     masses: np.ndarray
#         Array of stellar masses (solar mass)
#     masserr: np.ndarray
#         Array of sigma_mass (solar mass)
#     radii: np.ndarray
#         Array of stellar radii (solar radii)
#     raderr: np.ndarray
#         Array of sigma_radius (solar radii)

#     Returns
#     -------
#     rho_dist: np.ndarray
#         Array of density distributions for each star in kg/m^3
#         Each element length 1000
#     mass_dist: np.ndarray
#         Array of symmetric Gaussian mass distributions for each star in kg
#         Each element length 1000
#     rad_dist: np.ndarray
#         Array of symmetric Gaussian radius distributions for each star in m
#         Each element length 1000
#     """

#     smass_kg = 1.9885e30  # Solar mass (kg)
#     srad_m = 696.34e6     # Solar radius (m)

#     rho_dist = np.zeros((ntargs, 1000))
#     mass_dist = np.zeros((ntargs, 1000))
#     rad_dist = np.zeros((ntargs, 1000))

#     for star in tqdm(range(ntargs)):

#         rho_temp = np.zeros(1000)
#         mass_temp = np.zeros(1000)
#         rad_temp = np.zeros(1000)

#         mass_temp = np.random.normal(masses[star]*smass_kg, masserr[star]*smass_kg, 1000)
#         rad_temp = np.random.normal(radii[star]*srad_m, raderr[star]*srad_m, 1000)

#         #Add each density point to rho_temp (for each star)
#         for point in range(len(mass_temp)):
#             rho_temp[point] = density(mass_temp[point], rad_temp[point])

#         rho_dist[star] = rho_temp
#         mass_dist[star] = mass_temp
#         rad_dist[star] = rad_temp


#     return rho_dist, mass_dist, rad_dist

In [185]:
# def density(mass, radius):
#     """Get density of sphere given mass and radius.

#     Parameters
#     ----------
#     mass: float
#         Mass of sphere (kg)
#     radius: float
#         Radius of sphere (m)

#     Returns
#     rho: float
#         Density of sphere (kg*m^-3)
#     """

#     rho = mass/((4.0/3.0)*np.pi*radius**3)
#     return rho

In [186]:
import photoeccentric as ph

In [187]:
rho, mass, radius = ph.find_density_dist_symmetric(ntargs, [mstar], [mstar_err], [rstar], [rstar_err])

100%|██████████| 1/1 [00:00<00:00, 317.97it/s]


In [188]:
np.mean(rho)

6878.358708246579

In [189]:
np.mean(mass)

9.654111657058412e+29

In [190]:
np.mean(radius)

323769284.25628257

In [191]:
np.savetxt("jan29_21_rhos_" + '10388286' + ".csv", rho, delimiter=',')
#masspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")
#radiuspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")

In [192]:
mstar

0.485084227431942

In [193]:
rstar

0.46372314366762124